<a href="https://colab.research.google.com/github/Guiniel/Pasantia-IA-CNTM/blob/main/cargarModelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Instalamos los mismos paquetes de antes**

In [1]:
!pip install -q pyyaml h5py
!pip install git+https://github.com/tensorflow/docs

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


^C
Note: you may need to restart the kernel to use updated packages.


### **Importamos los paquetes a usar**

In [ ]:
import joblib
import pandas as pd
import numpy as np
import os
from IPython.display import clear_output
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

np.set_printoptions(precision=4)

### **Cargamos el dataset de ejemplo para realizar mis aproximaciones**

In [ ]:
data_pd = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/datasets/Evaluacion.csv")


### **Guardamos las columnas que no necesitamos y nos quedamos con las necesarias para poder hacer la prediccion**

In [ ]:
columns = ['Task_ID','Task_Name','Status', 'Start_Date','Due_Date','Time_Estimate','Time_Logged','Lists','Año','Q']
saved_columns = data_pd[columns]
data_pd = data_pd.drop(['Task_ID'], axis=1).drop(['Task_Name'], axis=1).drop(['Status'], axis=1).drop(['Start_Date'], axis=1).drop(['Due_Date'], axis=1).drop(['Time_Estimate'], axis=1).drop(['Time_Logged'], axis = 1).drop(['Año'],axis=1).drop(['Lists'], axis=1).drop(['Q'], axis=1)


### **Aqui hay funciones para par de metricas y para cargar el modelo**

In [ ]:
def cargar_modelo(ruta_archivo):
    nombre_archivo, extension = os.path.splitext(ruta_archivo)
    if extension == '.pkl':
        return joblib.load(ruta_archivo)
    elif extension == '.h5':
        return keras.models.load_model(ruta_archivo, custom_objects={'median_relative_error': median_relative_error, 'r_squared': r_squared})
    else:
        raise ValueError("Extensión de archivo no compatible.")
def median_relative_error(y_true, y_pred):
    error = tf.math.abs(y_true - y_pred) / tf.maximum(tf.math.abs(y_true), 1e-7)
    return tf.keras.backend.mean(error)

def r_squared(y_true, y_pred):
    SS_res =  tf.reduce_sum(tf.square(y_true - y_pred))
    SS_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))
    return 1 - SS_res/(SS_tot + tf.keras.backend.epsilon())


### **Aqui cargamos el modelo, realizamos las predicciones para el conjunto de datos cargados y lo exportamos con el mismo formato con el que entra**

### **Se puede elegir entre 6 modelos diferentes y el excel generado se crea en la carpeta de trabajo con el nombre del modelo**

In [ ]:
model_directory = '/content/drive/MyDrive/Colab Notebooks/modelos'
model_files = os.listdir(model_directory)

while True:
    clear_output(wait=True)

    print("Modelos disponibles:")
    for i, model_file in enumerate(model_files):
        print(f"{i+1}. {model_file}")

    print("0. Salir")

    # Pide al usuario que seleccione un modelo o salir
    selected_option = input("Por favor, selecciona un modelo o ingresa '0' para salir: ")

    if selected_option == '0':
        print("Saliendo del programa...")
        break  # Sale del bucle si el usuario elige salir

    try:
        selected_model_index = int(selected_option) - 1

        #Verifica si la selección está dentro del rango de modelos disponibles
        if selected_model_index < 0 or selected_model_index >= len(model_files):
            print("Por favor, selecciona una opción válida.")
            continue

        selected_model_path = os.path.join(model_directory, model_files[selected_model_index])
        print(f"Cargando modelo desde {selected_model_path}...")

        # Cargamos el modelo
        model = cargar_modelo(selected_model_path)

        result = model.predict(data_pd).flatten()

        # Juntamos la tabla nuevamente
        table = pd.concat([saved_columns, data_pd], axis=1)

        print(table)
        # Ordenamos los valores

        table = table[['Task_ID', 'Task_Name','Assignee', 'Status', 'Start_Date', 'Due_Date', 'Esfuerzo_Aproximado', 'Time_Estimate', 'Time_Logged','Año','Q','Mes','Lists']]
        # Agregamos los valores de prediccion
        table['Time_Estimate'] = result

        model_name = os.path.splitext(model_files[selected_model_index])[0]
        excel_filename = f"prediccion_{model_name}.xlsx"

        # Exportamos a excel
        table.to_excel(excel_filename, index = False)
    except ValueError:
        print("Por favor, ingresa un número válido.")
        input("Pulsa cualquier tecla para continuar...")